In [1]:
import shutil

# importing the sys module
import sys        

# appending the directory of mod.py
# in the sys.path list
sys.path.append('../')   

import json
import os
import random
from collections import defaultdict

import torch
from PIL import Image
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.optim import Adam
import numpy as np
from sklearn.model_selection import train_test_split

from datetime import datetime


import diff_pipeline4__config as cfg

from Common.Datasets.Morph2.data_parser import DataParser
from Common.Datasets.Morph2.Morph2ClassifierDataset2 import Morph2ClassifierDataset2
from Common.Datasets.Morph2.Morph2ClassifierDataset3 import Morph2ClassifierDataset3
from diff_pipeline4__train import train_diff_cls_model_iter

from Common.Optimizers.RangerLars import RangerLars
from Common.Schedulers.GradualWarmupScheduler import GradualWarmupScheduler


from diff_pipeline4__model import DiffPipeline4Model, DiffPipeline4DeepModel


from Common.Pipelines.AgePredictSingleRefPipeline import AgePredictSingleRefEnhancedPipeline #AgePredictSingleRefPipeline
#from Common.Analysis.DiffAnalysisMethods import diff_pipeline_confusion_matrix_analysis
from Common.Analysis.DiffAnalysisMethods import evaluate_pipeline_clean_enhanced


from tqdm import tqdm


from Common.Models.UnifiedClassificaionAndRegressionAgeModel import UnifiedClassificaionAndRegressionAgeModel
from Common.Models.transformer import *
from Common.Models.unified_transformer_model import AgeTransformer

from sklearn import utils


if __name__ == "__main__":

    torch.manual_seed(cfg.RANDOM_SEED)
    np.random.seed(cfg.RANDOM_SEED)
    random.seed(cfg.RANDOM_SEED)

    if cfg.USE_GPU:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("cpu")

    print(device)

    torch.cuda.empty_cache()





    # Load data
    data_parser = DataParser('../Common/Datasets/Morph2/aligned_data/aligned_dataset_with_metadata_uint8.hdf5', small_data=cfg.SMALL_DATA)
    data_parser.initialize_data()

    x_train, y_train, x_test, y_test = data_parser.x_train,	data_parser.y_train, data_parser.x_test, data_parser.y_test,
    if cfg.RANDOM_SPLIT:
        all_images = np.concatenate((x_train, x_test), axis=0)
        all_labels = np.concatenate((y_train, y_test), axis=0)

        x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=cfg.TEST_SIZE_FOR_RS, random_state=cfg.RANDOM_SEED)

    face2emb_arr_trn_r = np.load('face2emb_arr_trn.npy', allow_pickle=True)
#     import pdb
#     pdb.set_trace()
    #face2emb_arr_trn_r = face2emb_arr_trn_r.item()
    
    face2emb_arr_vld_r = np.load('face2emb_arr_vld.npy', allow_pickle=True)
    #face2emb_arr_vld_r = face2emb_arr_vld_r.item()
    
    # shuffle basic aligned test
    # x_test_shuffled
    # y_test_shuffled
    

    with open("im2age_map_test.json", 'r') as im2age_map_test_f:
        im2age_map_test = json.load(im2age_map_test_f)

#     transf_age_pred_model = transforms.Compose([
#             transforms.RandomResizedCrop(224, (0.9, 1.0)),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(
#                 brightness=0.1,
#                 contrast=0.1,
#                 saturation=0.1,
#                 hue=0.1
#             )], p=0.5),
#             transforms.RandomApply([transforms.RandomAffine(
#                 degrees=10,
#                 translate=(0.1, 0.1),
#                 scale=(0.9, 1.1),
#                 shear=5,
#                 resample=Image.BICUBIC
#             )], p=0.5),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#             transforms.RandomErasing(p=0.5)
#         ])



#################################
# transformsa with less random
###################
#     transf_diff_model = transforms.Compose([
#             transforms.Resize(224), # just for testing
#     #          transforms.RandomResizedCrop(224, (0.9, 1.0)),
#     #          transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(
#                 brightness=0.1,
#                 contrast=0.1,
#                 saturation=0.1,
#                 hue=0.1
#             )], p=0.5),
#     #         transforms.RandomApply([transforms.RandomAffine(
#     #             degrees=10,
#     #             translate=(0.1, 0.1),
#     #             scale=(0.9, 1.1),
#     #             shear=5,
#     #             resample=Image.BICUBIC
#     #         )], p=0.5),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     #        transforms.RandomErasing(p=0.5)
#     ])


    transf_diff_model = transforms.Compose([
            transforms.RandomResizedCrop(224, (0.9, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(
                brightness=0.1,
                contrast=0.1,
                saturation=0.1,
                hue=0.1
            )], p=0.5),
            transforms.RandomApply([transforms.RandomAffine(
                degrees=10,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=5,
                resample=Image.BICUBIC
            )], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.5)
        ])


    test_ds = Morph2ClassifierDataset3( #Morph2ClassifierDataset Morph2ClassifierDataset2
        x_test,
        y_test,
        cfg.MIN_AGE,
        cfg.AGE_INTERVAL,
        transform=transforms.Compose([
            transforms.RandomResizedCrop(224, (0.9, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(
                brightness=0.1,
                contrast=0.1,
                saturation=0.1,
                hue=0.1
            )], p=0.5),
            transforms.RandomApply([transforms.RandomAffine(
                degrees=10,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=5,
                resample=Image.BICUBIC
            )], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.5)
        ]),
        copies=cfg.NUM_OF_COPIES_AGE_PREDICT
    )

    image_datasets = {
        'train': [],
        'val': test_ds
    }

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    data_loaders = {
        'train': None, #DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=True, drop_last=True),
        'val': DataLoader(test_ds, batch_size=1, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=False)
    }


    num_classes = int((cfg.MAX_AGE - cfg.MIN_AGE) / cfg.AGE_INTERVAL + 1)


    # create model and parameters
    if cfg.DEEP_MODEL:
        model = DiffPipeline4DeepModel
    else:
        model = DiffPipeline4Model

    print("testing set size: " + str(len(test_ds)))
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_30_10_2022_17_23_12' # trained in range 0-3 with close embs (from 25 percentile), unfreeze backbone in epoch 5
    #model_filename = 'weights_11064_1.3333.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_04_11_2022_04_00_05' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_6915_1.3252.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_18_11_2022_11_02_25' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_5532_1.3246.pt'

    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_04_11_2022_21_42_24' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 10
    #model_filename = 'weights_16596_1.3250.pt'

    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_06_11_2022_11_38_47' # trained in range 0-4 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_11064_1.6606.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_08_11_2022_01_46_52' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_4200_1.3240.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_10_11_2022_09_20_42' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_2766_1.3247.pt'
    


    
    pipeline = AgePredictSingleRefEnhancedPipeline(
                        device,
                        num_classes, 
                        cfg.AGE_INTERVAL, 
                        cfg.MIN_AGE, 
                        cfg.MAX_AGE, 
                        cfg.AGE_RADIUS,
                        cfg.MID_FEATURE_SIZE_AGE_PREDICT, 
                        x_train,
                        y_train,
                        0, #cfg.DEEP, 
                        0, #cfg.NUM_OF_REFERENCES, 
                        0, #cfg.DIFF_MODEL_CONFIG_TYPE, 
                        0, #cfg.ADDED_EMBED_LAYER_SIZE,
                        0, #cfg.DIFF_EMBED_LAYER_SIZE,
                        transf_diff_model,
                        cfg.NUM_OF_COPIES_AGE_PREDICT,
                        UnifiedClassificaionAndRegressionAgeModel,
                        TransformerModel,
                        AgeTransformer,
                        model, #AgeDiffModelNonOpt, #AgeDiffModel,
                        True, #cfg.IS_ORDINAL_REG,
                        no_age_transformer_init=False,
                        cfg=cfg,
                        age_diff_model_path_arg=model_dir,
                        age_diff_model_file_name_arg=model_filename,
                        embs_trn=face2emb_arr_trn_r,
                        embs_vld=face2emb_arr_vld_r)


    #diff_pipeline_confusion_matrix_analysis(pipeline, data_loaders['val'], device, dataset_sizes['val'], is_ordinal_reg=True)


    if cfg.CHECK_DIFF_BASED:
        mae_on_test = evaluate_pipeline_clean_enhanced(pipeline, 
                                                        data_loaders['val'], 
                                                        device, 
                                                        dataset_sizes['val'], 
                                                        bypass_diff=False, 
                                                        compare_post_round=False, 
                                                        range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, 
                                                        range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, 
                                                        im2age_map_test=im2age_map_test, 
                                                        check_against_pred_db=False)
    else:
        mae_on_test = evaluate_pipeline_clean_enhanced(pipeline, 
                                                    data_loaders['val'], 
                                                    device, 
                                                    dataset_sizes['val'], 
                                                    bypass_diff=True, 
                                                    compare_post_round=False, 
                                                    range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, 
                                                    range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, 
                                                    im2age_map_test=im2age_map_test, 
                                                    check_im2age_map_test=False,
                                                    create_im2age_map_test=False, 
                                                    check_against_pred_db=False)
        #mae_on_test = evaluate_pipeline_clean_enhanced(pipeline, data_loaders['val'], device, dataset_sizes['val'], bypass_diff=True, compare_post_round=False, range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, im2age_map_test=im2age_map_test)



cuda:0


C:\Users\proj\.conda\envs\my_thesis_py38\lib\site-packages\torchvision\transforms\transforms.py:1361: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


testing set size: 10606
Using multiple GPUs (2)
Using multiple GPUs (2)
running on validation set...


D:\projects\age_estimation\DiffPipeline4\..\Common\Models\transformer.py:127: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = nn.Softmax()(classification_logits)


- IN RANGE:  10
- IN RANGE mae sum :  tensor(21.0650)
- OUT OF RANGE:  41
- OUT OF RANGE mae sum :  tensor(251.1512)
- total:  51
- total mae sum :  tensor(272.2162)
progress : 0.48%
current MAE (in range): 2.1065
current MAE (out of range): 6.1256
current MAE (total): 5.3376
current MAE diff (in range): 2.1065
current MAE diff (out of range): 6.1256
- IN RANGE:  72
- IN RANGE mae sum :  tensor(122.8435)
- OUT OF RANGE:  79
- OUT OF RANGE mae sum :  tensor(439.1794)
- total:  151
- total mae sum :  tensor(562.0230)
progress : 1.42%
current MAE (in range): 1.7062
current MAE (out of range): 5.5592
current MAE (total): 3.7220
current MAE diff (in range): 1.7062
current MAE diff (out of range): 5.5592
- IN RANGE:  139
- IN RANGE mae sum :  tensor(227.9492)
- OUT OF RANGE:  112
- OUT OF RANGE mae sum :  tensor(590.3959)
- total:  251
- total mae sum :  tensor(818.3453)
progress : 2.37%
current MAE (in range): 1.6399
current MAE (out of range): 5.2714
current MAE (total): 3.2603
current MAE

- IN RANGE:  1388
- IN RANGE mae sum :  tensor(1939.4026)
- OUT OF RANGE:  963
- OUT OF RANGE mae sum :  tensor(4947.9448)
- total:  2351
- total mae sum :  tensor(6887.3408)
progress : 22.17%
current MAE (in range): 1.3973
current MAE (out of range): 5.1381
current MAE (total): 2.9295
current MAE diff (in range): 1.3973
current MAE diff (out of range): 5.1381
- IN RANGE:  1445
- IN RANGE mae sum :  tensor(2026.7404)
- OUT OF RANGE:  1006
- OUT OF RANGE mae sum :  tensor(5182.3022)
- total:  2451
- total mae sum :  tensor(7209.0361)
progress : 23.11%
current MAE (in range): 1.4026
current MAE (out of range): 5.1514
current MAE (total): 2.9413
current MAE diff (in range): 1.4026
current MAE diff (out of range): 5.1514
- IN RANGE:  1519
- IN RANGE mae sum :  tensor(2128.1367)
- OUT OF RANGE:  1032
- OUT OF RANGE mae sum :  tensor(5317.4492)
- total:  2551
- total mae sum :  tensor(7445.5791)
progress : 24.05%
current MAE (in range): 1.4010
current MAE (out of range): 5.1526
current MAE (

- IN RANGE:  2763
- IN RANGE mae sum :  tensor(3889.6477)
- OUT OF RANGE:  1888
- OUT OF RANGE mae sum :  tensor(9652.7119)
- total:  4651
- total mae sum :  tensor(13542.3896)
progress : 43.85%
current MAE (in range): 1.4078
current MAE (out of range): 5.1127
current MAE (total): 2.9117
current MAE diff (in range): 1.4078
current MAE diff (out of range): 5.1127
- IN RANGE:  2834
- IN RANGE mae sum :  tensor(3992.7051)
- OUT OF RANGE:  1917
- OUT OF RANGE mae sum :  tensor(9791.2529)
- total:  4751
- total mae sum :  tensor(13783.9834)
progress : 44.80%
current MAE (in range): 1.4089
current MAE (out of range): 5.1076
current MAE (total): 2.9013
current MAE diff (in range): 1.4089
current MAE diff (out of range): 5.1076
- IN RANGE:  2911
- IN RANGE mae sum :  tensor(4109.8506)
- OUT OF RANGE:  1940
- OUT OF RANGE mae sum :  tensor(9897.2383)
- total:  4851
- total mae sum :  tensor(14007.1113)
progress : 45.74%
current MAE (in range): 1.4118
current MAE (out of range): 5.1017
current M

- IN RANGE:  4325
- IN RANGE mae sum :  tensor(6067.1787)
- OUT OF RANGE:  2626
- OUT OF RANGE mae sum :  tensor(13235.9814)
- total:  6951
- total mae sum :  tensor(19303.1934)
progress : 65.54%
current MAE (in range): 1.4028
current MAE (out of range): 5.0404
current MAE (total): 2.7770
current MAE diff (in range): 1.4028
current MAE diff (out of range): 5.0404
- IN RANGE:  4407
- IN RANGE mae sum :  tensor(6167.1089)
- OUT OF RANGE:  2644
- OUT OF RANGE mae sum :  tensor(13329.5576)
- total:  7051
- total mae sum :  tensor(19496.6992)
progress : 66.48%
current MAE (in range): 1.3994
current MAE (out of range): 5.0414
current MAE (total): 2.7651
current MAE diff (in range): 1.3994
current MAE diff (out of range): 5.0414
- IN RANGE:  4486
- IN RANGE mae sum :  tensor(6271.8110)
- OUT OF RANGE:  2665
- OUT OF RANGE mae sum :  tensor(13436.0938)
- total:  7151
- total mae sum :  tensor(19707.9258)
progress : 67.42%
current MAE (in range): 1.3981
current MAE (out of range): 5.0417
curren

- IN RANGE:  6043
- IN RANGE mae sum :  tensor(8262.5557)
- OUT OF RANGE:  3208
- OUT OF RANGE mae sum :  tensor(16154.1309)
- total:  9251
- total mae sum :  tensor(24416.7031)
progress : 87.22%
current MAE (in range): 1.3673
current MAE (out of range): 5.0356
current MAE (total): 2.6394
current MAE diff (in range): 1.3673
current MAE diff (out of range): 5.0356
- IN RANGE:  6116
- IN RANGE mae sum :  tensor(8342.5449)
- OUT OF RANGE:  3235
- OUT OF RANGE mae sum :  tensor(16272.4209)
- total:  9351
- total mae sum :  tensor(24614.9844)
progress : 88.17%
current MAE (in range): 1.3641
current MAE (out of range): 5.0301
current MAE (total): 2.6323
current MAE diff (in range): 1.3641
current MAE diff (out of range): 5.0301
- IN RANGE:  6193
- IN RANGE mae sum :  tensor(8439.4150)
- OUT OF RANGE:  3258
- OUT OF RANGE mae sum :  tensor(16378.4531)
- total:  9451
- total mae sum :  tensor(24817.8848)
progress : 89.11%
current MAE (in range): 1.3627
current MAE (out of range): 5.0271
curren